In [2]:
#! python3
import os
import pandas as pd
import numpy as np
import gdown
from datetime import datetime

# display more than 5 columns
desired_width = 320
pd.set_option('display.width', desired_width)
np.set_printoptions(linewidth=desired_width)
pd.set_option('display.max_columns', 10)
gdown.download(id='1QmNtncaokay60GotFuVV953fd9TxCbDr')
data = pd.read_csv('COVID-19_Vaccinations_in_the_United_States_County.csv', header=[0])

Downloading...
From: https://drive.google.com/uc?id=1QmNtncaokay60GotFuVV953fd9TxCbDr
To: C:\Users\sara2\modern data analytics\MDA_final_Sweden\COVID-19_Vaccinations_in_the_United_States_County.csv
100%|███████████████████████████████████████████████████████████████████████████████| 349M/349M [00:57<00:00, 6.07MB/s]
C:\Users\sara2\AppData\Local\Temp\ipykernel_2216\640108589.py:14: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('COVID-19_Vaccinations_in_the_United_States_County.csv', header=[0])


In [3]:
data.head()

,Date,FIPS,MMWR_week,Recip_County,Recip_State,...,Census2019_5PlusPop,Census2019_5to17Pop,Census2019_12PlusPop,Census2019_18PlusPop,Census2019_65PlusPop
0,05/03/2022,01013,18,Butler County,AL,...,18348.0,3174.0,16675.0,15174.0,4049.0
1,05/03/2022,01045,18,Dale County,AL,...,45899.0,7915.0,41540.0,37984.0,8502.0
2,05/03/2022,05065,18,Izard County,AR,...,13064.0,1883.0,12084.0,11181.0,3558.0
3,05/03/2022,12043,18,Glades County,FL,...,13407.0,1584.0,12604.0,11823.0,3808.0
4,05/03/2022,13179,18,Liberty County,GA,...,55036.0,11002.0,48507.0,44034.0,5908.0


In [4]:
# Choosing only columns of interest
# Recip_State -> State
# Series_Complete_Yes -> Total number of people who are fully vaccinated
# 'Booster_Doses' -> Total number of people who are fully vaccinated and have received a booster (or additional) dose
data = data[['Date','Recip_County', 'MMWR_week', 'Recip_State', 'Series_Complete_Yes',
             'Booster_Doses']]
# cap for date (01/04/2022)
data['Date'] = pd.to_datetime(data['Date'], format="%m/%d/%Y")
data = data[data['Date'] < '2022-04-01']

In [5]:
# Checking the numer of NAs
NAs = data.columns[data.isnull().any()].tolist()
num = data.isna().sum().sum()
print('Missing values: '+ str(NAs)+'. Total number: '+str(num))
#for i in NAs:
#    print(i + '\t\t' + str(data[i].isnull().sum()))

Missing values: ['Series_Complete_Yes', 'Booster_Doses']. Total number: 1209670


In [6]:
data = data.fillna(value='NA') # missing values in the data file are blank,changing it to NA

In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1555611 entries, 108366 to 1663976
Data columns (total 6 columns):
 #   Column               Non-Null Count    Dtype         
---  ------               --------------    -----         
 0   Date                 1555611 non-null  datetime64[ns]
 1   Recip_County         1555611 non-null  object        
 2   MMWR_week            1555611 non-null  int64         
 3   Recip_State          1555611 non-null  object        
 4   Series_Complete_Yes  1555611 non-null  object        
 5   Booster_Doses        1555611 non-null  object        
dtypes: datetime64[ns](1), int64(1), object(4)
memory usage: 83.1+ MB


In [8]:
#459640, 12/15/2021 is the first row when booster doses start being recorded
data.loc[(data.Date<'2021-12-15'),'Booster_Doses']=0
data.tail()

,Date,Recip_County,MMWR_week,Recip_State,Series_Complete_Yes,Booster_Doses
1663972,2020-12-13,Boulder County,51,CO,0.0,0
1663973,2020-12-13,Kit Carson County,51,CO,0.0,0
1663974,2020-12-13,Floyd County,51,IA,0.0,0
1663975,2020-12-13,Brantley County,51,GA,0.0,0
1663976,2020-12-13,Mecosta County,51,MI,0.0,0


In [11]:
sort_data=data.copy(deep=True)
sort_data.sort_values(by=['Recip_County','Date'],inplace=True, ascending=True)

In [12]:
sort_data

,Date,Recip_County,MMWR_week,Recip_State,Series_Complete_Yes,Booster_Doses
1663844,2020-12-13,Abbeville County,51,SC,0.0,0
1658104,2020-12-14,Abbeville County,51,SC,0.0,0
1657350,2020-12-15,Abbeville County,51,SC,0.0,0
1654046,2020-12-16,Abbeville County,51,SC,0.0,0
1648288,2020-12-17,Abbeville County,51,SC,0.0,0
...,...,...,...,...,...,...
123551,2022-03-27,Ziebach County,13,SD,915.0,305.0
121143,2022-03-28,Ziebach County,13,SD,915.0,305.0
115879,2022-03-29,Ziebach County,13,SD,915.0,305.0
112858,2022-03-30,Ziebach County,13,SD,916.0,306.0


In [13]:
''' We can replace NA with mean or mode. However if we know the number of boosters/series in a particular place at the previous date
we can imputing the missing value with the value from the next day. To achieve it we need to check present day and the day
before'''
for i in range(1,len(sort_data)):
    if sort_data.iloc[i,-2]=='NA':
        if sort_data.iloc[i,1]==sort_data.iloc[i-1,1]:
            sort_data.iloc[i,-2] = sort_data.iloc[i-1,-2]
        else:
            sort_data.iloc[i,-2] = 0 

for i in range(1,len(sort_data)):
    if sort_data.iloc[i,-1]=='NA':
        if sort_data.iloc[i,1]==sort_data.iloc[i-1,1]:
            sort_data.iloc[i,-1] = sort_data.iloc[i-1,-1]
        else:
            sort_data.iloc[i,-1] = 0

In [14]:
sort_data[sort_data=='NA'].count()

Date                   0
Recip_County           0
MMWR_week              0
Recip_State            0
Series_Complete_Yes    0
Booster_Doses          0
dtype: int64

In [15]:
sort_data.head()

,Date,Recip_County,MMWR_week,Recip_State,Series_Complete_Yes,Booster_Doses
1663844,2020-12-13,Abbeville County,51,SC,0.0,0
1658104,2020-12-14,Abbeville County,51,SC,0.0,0
1657350,2020-12-15,Abbeville County,51,SC,0.0,0
1654046,2020-12-16,Abbeville County,51,SC,0.0,0
1648288,2020-12-17,Abbeville County,51,SC,0.0,0


In [16]:
abbrev = pd.read_csv("RawData/state_fips.csv")
abbrev = abbrev.drop('fips', axis=1)
abbrev = abbrev.rename(columns={'Postal Code':'Code'})
abbrev = abbrev.rename(columns={'Name':'State'})
sort_data = sort_data.rename(columns={'Recip_State':'Code'})
data_vac = pd.merge(sort_data,abbrev, on='Code')
data_vac = data_vac.drop('Code', axis=1)
data_vac_sort = data_vac.sort_values(by=['Date','Recip_County'],inplace=True, ascending=False)
data_vac[data_vac.Date<'2021-01-01']
data_vac.to_csv("PreprocessedData/vaccine_per_county.csv",sep=',',index=False)